In [ ]:
%reset -f
import matplotlib.pyplot as plt
from algorithms import *
from typing import Callable
from pathlib import Path
import os

Define different numbers of selection points and datasets to interpolate

In [ ]:
NUMBERS_OF_SELECTION_POINTS = [7, 13, 23, 31, 47, 103]
DATASETS = os.listdir('../data')

Define plotting function that interpolates data and plots it

In [ ]:
def plot_interpolation(
        handler: any,
        data: list[tuple[float, float]],
        select_strategy: Callable[[float, float, int], list[float]],
        select_points_number: int,
        interpolation_method: Callable[[list[tuple[float, float]], list[float]], list[float]],
        title: str
    ) -> None:
    interpolation_nodes = select_points_by_strategy(data, select_strategy, select_points_number)
    x_data = get_x_values(data)
    y_interpolated = interpolation_method(interpolation_nodes, x_data)

    handler.scatter(get_x_values(interpolation_nodes), get_y_values(interpolation_nodes), color='red', s=10)
    handler.plot(x_data, get_y_values(data), color='blue')
    handler.plot(x_data, y_interpolated, color='orange')

    handler.title.set_text(title)
    handler.set_xlabel('Distance, m')
    handler.set_ylabel('Height, m')
    handler.legend(['Selection points', 'Original function', 'Interpolated function'])
    handler.set_yscale('symlog')

Plot interpolation results for each dataset and each interpolation method

In [ ]:
for dataset in DATASETS:
    nodes = read_data("../data/" + dataset)
    data_name = dataset.replace(".csv", "").replace(".txt", "")
    Path(f'../plots/{data_name}/').mkdir(parents=True, exist_ok=True)

    # Original function
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 6)
    ax.set_title(f'{dataset} - Original function')
    ax.set_xlabel('Distance, m')
    ax.set_ylabel('Height, m')
    ax.plot(get_x_values(nodes), get_y_values(nodes))
    ax.set_yscale('symlog')
    fig.savefig(f'../docs/plots/{data_name}/original.png')

    # Linearly spaced nodes
    fig, axs = plt.subplots(3, 2)
    fig.set_size_inches(20, 18)
    fig.suptitle(f'{dataset} - Lagrange interpolation with linearly spaced nodes')
    for i in range(len(NUMBERS_OF_SELECTION_POINTS)):
        row, col = i//2, i%2
        plot_interpolation(axs[row, col], nodes, linspace, NUMBERS_OF_SELECTION_POINTS[i], lagrange_interpolation, f'{NUMBERS_OF_SELECTION_POINTS[i]} selection points')
    fig.savefig(f'../docs/plots/{data_name}/linspace_nodes.png')

    # Chebyshev nodes
    fig, axs = plt.subplots(3, 2)
    fig.set_size_inches(20, 18)
    fig.suptitle(f'{dataset} - Lagrange interpolation with Chebyshev nodes')
    for i in range(len(NUMBERS_OF_SELECTION_POINTS)):
        row, col = i//2, i%2
        plot_interpolation(axs[row, col], nodes, get_chebyshev_nodes, NUMBERS_OF_SELECTION_POINTS[i], lagrange_interpolation, f'{NUMBERS_OF_SELECTION_POINTS[i]} selection points')
    fig.savefig(f'../docs/plots/{data_name}/chebyshev_nodes.png')

    # Cubic spline interpolation
    fig, axs = plt.subplots(3, 2)
    fig.set_size_inches(20, 18)
    fig.suptitle(f'{dataset} - Cubic spline interpolation')
    for i in range(len(NUMBERS_OF_SELECTION_POINTS)):
        row, col = i//2, i%2
        plot_interpolation(axs[row, col], nodes, linspace, NUMBERS_OF_SELECTION_POINTS[i], cubic_spline_interpolation, f'{NUMBERS_OF_SELECTION_POINTS[i]} selection points')
    fig.savefig(f'../docs/plots/{data_name}/cubic_spline.png')